#### RPA_day5 정리  

파이썬으로 텍스트 또는 html형식으로 이메일 보내기  
1. SMTP('smtp 서버 dns', port) 객체 생성, SMTP_SSL('smtp 서버 dns', port) 객체 생성  
2. ehlo()   (SMTP 객체의 경우.starttls())
3. login()
4. MIMEText ('메일내용', _subtype, _charset=)
    MIMEText ['Suject']
    MIMEText ['From']
    MIMEText ['To']
5. sendmail(sender, receiver, MIMEText객체를 문자열반환)
6. SMTP 서버 접속 종료 quit()


첨부파일 MIMEBase('multipart')     
         MIMEBase ['Suject']         
         MIMEBase ['From']     
         MIMEBase ['To']      
         MIMEBase.attach(MIMEText 객체 or MIMEImage 객체 or MIMEAudio객체 or MIMEApplication)   

메일 전송 프로토콜 헤더 설정 추가    
MIMEText 객체 or MIMEImage 객체 or MIMEAudio객체 or MIMEApplication객체의   
add_header('content-disposition', attachment, filename=, )  

#### 자동으로 이메일 보내기 ( 이메일 관리 )

메일을 가져오기 위해서 사용하는 프로토콜 POP3, IMAP

• POP3(Post Office Protocol 3) – 이메일 서버에서 메일을 가져오기 위해서 사용하는 프로토콜    
한번 서버로부터 내려 받은 이메일은 기본적으로 서버에서 삭제되고 서버가 더 이상 관리하지 않는다    

• IMAP(Internet Message Access Protocol) - 이메일 서버에서 메일을 가져오기 위해서 사용하는 프로토콜    
여러 군데에서 이메일 서버로부터 메일을 읽는 것을 허용한다.    
한 클라이언트에서 메일을 삭제하면 서버에서도 삭제되고 다른 클라이언트 컴퓨터에서도 동기화하게 된다.

In [1]:
! pip install imapclient
! pip install pyzmail36

  Created wheel for pyzmail36: filename=pyzmail36-1.0.4-py3-none-any.whl size=37083 sha256=515f66aee75a85b7387cad68dd82e16796375af04e8efeae77f49c5e173fcba5
  Stored in directory: c:\users\fbvl\appdata\local\pip\cache\wheels\31\94\b8\b899e4c384383f0032cd71b41ede5931024830aabf81acb220
Successfully built pyzmail36


• 메일은 폴더별로 관리하게 되어 있어, 작업 전에 폴더를 선택해야 한다   
예]gmail의 ‘받은 편지함’의 이름은 ‘INBOX’

• list_folder() – 로그인되어 있는 상태에서 폴더 목록을 얻어낼 수 있다

• select_folder() - 작업할 폴더의 선택, readonly 인수가 True이면 메일을 읽기만 할 수 있고 False이면 읽기와 삭제가 가능  

• search() – 검색 기준을 리스트나 튜플과 같은 시퀀스 형식의 인수로 받고, 처리 결과로 메시지 아이디(정수)의 리스트를 반환

구글 : imap.gmail.com   
네이버 : imap.naver.com    
다음 : imap.daum.net    
야후 : imap.mail.yahoo.com    
Hotmail : imap-mail.outlook.com   
네이트온 : imap.nate.com   

In [ ]:
#사용예시
imap.list_folders()[1:10]  #메일함 폴더에서 인덱싱해서 가져오기
imap.select_folder('INBOX', readonly=False); #파이썬으로 읽어도 실제 메일함에는 ‘읽은메일’로 표시되지 않음

In [36]:
import imapclient
import pyzmail

imap = imapclient.IMAPClient('imap.naver.com', ssl=True)
imap.login('', '')
#imap.logout()

imap.list_folders()[:]
imap.select_folder('INBOX', readonly=False) #받은 메일함 선택

uids = imap.search(['ALL']) # 받음 메일함으로부터 전체 메일 ID를 반환

print(len(uids))  #받은 메일함으로부터 전체 메일 수

uids[:5]  #이메일 아이디 리스트 5개 확인

uids = imap.search(['ON', '19-APR-2021']) # 문자열 날짜로 검색
print(uids)


1000
[51384, 51385, 51386, 51387, 51389, 51390, 51391, 51392, 51393, 51394, 51395]


In [37]:
from datetime import date, timedelta
uids = imap.search(['ON', date.today()])  #오늘 받은 메일 반환
print(uids)

uids = imap.search(['SINCE', date.today() - timedelta(days=7)])  #오늘부터 7일간 받은 메일 반환

uids = imap.search(['SINCE', date(2020, 10, 3), 'BEFORE', date(2020, 11, 3)]) # 지정된 날짜 범위로 검색

[51447, 51449, 51450, 51451, 51452, 51453, 51454]


In [39]:
uids = imap.search(['FROM', 'adrian']) # 보내는 사람 이름으로 검색
uids = imap.search(['FROM', 'sim11241124@gmail.com']) # 보내는 사람 이메일으로 검색
uids = imap.search(['SUBJECT', 'Sera'], 'utf-8') # 제목에 ''포함하고 있는 메일, 한글일 경우 인코딩 설정
uids = imap.search(['UNSEEN']) # 읽지 않은 메일
uids = imap.search(['FLAGGED']) # '중요(별표)' 플래그가 붙은 메일
uids = imap.search(['LARGER', 100000]) # 메시지 크기로 검색
uids = imap.search(['FROM', 'sim11241124@gmail.com', 'SUBJECT', 'Python', 'SINCE', date(2020, 1, 1), 'SMALLER',
100000, 'FLAGGED'])  #여러가지 조건을 조합해서 검색

In [49]:
#100k보다 작거나 500k보다 큰 메일 메시지의 메일 아이디 반환
#uids = imap.search(['NOT', ['LARGER', 100000, 'SMALLER', 500000]])
import imapclient
import pyzmail

imap = imapclient.IMAPClient('imap.gmail.com', ssl=True)
imap.login('', '')
#imap.logout()

uids = imap.select_folder('[Gmail]/전체보관함', readonly=False)
uids = imap.gmail_search('from:googleplay-noreply@google.com') # 송신자를 이용한 검색
uids = imap.gmail_search('to:sim11241124@gmail.com') # 수신자를 이용한 검색

이메일 관리 - 메일 삭제    
검색된 메일 삭제 - delete_message() #삭제 표시만  
물리적으로 삭제 - expunge()
• 특정 발신자 메일 삭제

In [46]:
uids = imap.search(['FROM', 'fbvl@naver.com']) # 송신자 이메일 주소로 온 메일 삭제
print('deleting {} messages'.format(len(uids)))
imap.delete_messages(uids)
imap.expunge()

deleting 0 messages


(b'Success', [])

In [53]:
#이메일 관리 – 광고 메일 삭제

import imapclient

imap = imapclient.IMAPClient('imap.gmail.com', ssl=True)
imap.login('sim11241124@gmail.com', 'Tprud11241124!')
uids = imap.select_folder('[Gmail]/전체보관함', readonly=False)
uids = imap.gmail_search('subject:광고')
print('deleting {} messages'.format(len(uids)))
imap.delete_messages(uids)
imap.expunge()
imap.logout()

deleting 5 messages


b'LOGOUT Requested'

In [ ]:
이메일 관리 – 광고 메일 삭제   
• fetch(message_set, message_parts) – 메일 본문 내용을 읽을 수 있게 해준다.    
주어진 여러 메시지 아이디의 메일을 한꺼번에 읽을 수 있다    
message_set : 메시지 아이디의 리스트    
message_parts : 메시지 부분으로 이메일에서 어떤 정보(제목, 본문 등)를 가져올지를 지시하는 문자열

In [64]:
import imapclient

imap = imapclient.IMAPClient('imap.gmail.com', ssl=True)
imap.login('', '')
uids = imap.select_folder('[Gmail]/전체보관함', readonly=False)

uids = imap.search(['FROM', 'Google'])
print(uids)
 #['BODY[]', 'FLAGS']=이메일에서 가져온 정보를 list로 전달
rmsgs = imap.fetch(uids, ['BODY[]', 'FLAGS']) 
print(type(rmsgs))

rmsgs[1536].keys()  #dict_keys([b'SEQ', b'FLAGS', b'BODY[]'])
rmsgs[1536][b'SEQ']
rmsgs[1536][b'FLAGS']
#rmsgs[1536][b'BODY[]'] #내용이 길어 주석처리



[1, 2, 3, 8, 42, 50, 58, 68, 73, 78, 82, 84, 86, 91, 95, 98, 102, 109, 114, 120, 126, 133, 138, 238, 241, 535, 663, 681, 757, 818, 848, 922, 936, 971, 1062, 1092, 1123, 1137, 1138, 1169, 1181, 1196, 1215, 1230, 1254, 1261, 1270, 1273, 1319, 1335, 1363, 1389, 1392, 1409, 1411, 1413, 1418, 1431, 1432, 1433, 1434, 1435, 1436, 1437, 1438, 1445, 1454, 1456, 1459, 1461, 1462, 1464, 1468, 1469, 1470, 1471, 1472, 1475, 1476, 1477, 1478, 1487, 1489, 1491, 1501, 1507, 1508, 1510, 1513, 1520, 1521, 1522, 1524, 1525, 1526, 1527, 1528, 1529, 1531, 1532, 1533, 1535, 1536, 1537, 1541, 1545, 1550, 1556, 1584, 1585, 1595, 1596, 1597, 1604, 1606, 1607, 1611]
<class 'collections.defaultdict'>


(b'\\Seen',)

이메일 관리 – 메일 내용 읽기    
• pyzmail(https://pypi.org/project/pyzmail)은 메일 본문을 분석하고, 이메일을 작성하고, 메일을 보내는 기능의 함수와 클래스를 제공한다.

In [67]:
import pyzmail

message = pyzmail.PyzMessage.factory(rmsgs[uids[0]][b'BODY[]'])  #본문 내용을 해석
message
message.get_subject()  #제목
message.get_addresses('from')  #발신
message.get_addresses('to')  #수신
message.get_addresses('cc')  #참조
message.get('Date')  #메일 보낸 시간
if message.text_part:  #본문 내용 텍스트
    print(message.text_part.get_payload().decode(message.text_part.charset))

 [image: Access Gmail on your mobile phone]
<http://www.google.com/intl/ko/mobile/default/mail.html>

컴퓨터로 받은편지함을 열어보는 시대는 지났습니다. 이제 휴대전화에서 Gmail을 사용하여 어디에서나 이메일에 액세스할 수 있습니다.
       휴대전화용 Gmail 다운로드
>><http://www.google.com/intl/ko/mobile/default/mail.html#utm_source=wel-eml&utm_medium=eml&utm_campaign=ko>



In [69]:
if message.html_part:  #본문 내용 html
    html = message.html_part.get_payload().decode(message.html_part.charset)

html[:200]

'<html>\r\n<font face="Arial, Helvetica, sans-serif">\r\n<p>\r\n  <a href="http://www.google.com/intl/ko/mobile/default/mail.html">\r\n<img src="https://www.google.com/intl/ko/mobile/images/phones.gif"\r\n      '

In [91]:
rmsgs = imap.fetch([42], ['BODY[]'])
message = pyzmail.PyzMessage.factory(rmsgs[42][b'BODY[]'])
len(message.mailparts)  #첨부파일 확인 속성 

part = message.mailparts[0]
part
part.type   #첨부파일의 MIME타입
part.charset   #첨부파일의 문자셋
part.get_payload().decode(part.charset)  #본문 내용, 한글로 인코딩

'[ Google Play ] (https://play.google.com/store/)\r\n\r\n안드로이드 게임과 앱\r\n\r\nMarch 27th, 2013\r\n\r\n#  베스트 인기 앱과\r\n게임 찾기\r\n\r\nGoogle Play의 인기 차트와 카테고리에서 가장 많이 다운로드 된 앱과 게임을  \r\n검색할 수 있습니다.\r\n\r\n    \r\n(http://clickserve.dartsearch.net/link/click?lid=43700003908951094&ds_s_kwgid=58700000055778857&ds_e_adid={creative}&ds_e_matchtype={ifsearch:search}{ifcontent:content}&ds_url_v=2)\r\n[\r\n\r\n앱 더보기\r\n\r\n](http://clickserve.dartsearch.net/link/click?lid=43700003908951094&ds_s_kwgid=58700000055778857&ds_e_adid={creative}&ds_e_matchtype={ifsearch:search}{ifcontent:content}&ds_url_v=2)\r\n\r\n\xa0\r\n\r\n##  안드로이드 추천 카테고리\r\n\r\n    \r\n(http://clickserve.dartsearch.net/link/click?lid=43700003908951096&ds_s_kwgid=58700000055778857&ds_e_adid={creative}&ds_e_matchtype={ifsearch:search}{ifcontent:content}&ds_url_v=2)\r\n    \r\n(http://clickserve.dartsearch.net/link/click?lid=43700003908951098&ds_s_kwgid=58700000055778857&ds_e_adid={creative}&ds_e_matchtype={ifsearch:search}{ifcontent:

In [119]:
imap = imapclient.IMAPClient('imap.gmail.com', ssl=True)
imap.login('', '')
uids = imap.select_folder('[Gmail]/전체보관함', readonly=False)

uids = imap.search(['FROM', 'fbvl@naver.com'])

#rmsgs = imap.fetch(uids, ['BODY[]', 'FLAGS']) 
print(uids)

rmsgs = imap.fetch([1635], ['BODY[]'])
message = pyzmail.PyzMessage.factory(rmsgs[1635][b'BODY[]'])
#len(message.mailparts)  #첨부파일 확인 속성 


#message.mailparts[0] 첫 파트 형식이 text/palin인 본문
#message.mailparts[2] 두 번재 파트 형식은 본문 내용과 동일한 내용의 text/html
part = message.mailparts[2] #세 번재 파트 형식은 첨부파일
print(part)
part.type  #image/png
cont = part.get_payload()
open(part.filename, 'wb').write(cont)  #첨부파일 이미지를 로컬 특정 디렉토리에 저장

[1635]
MailPart<image/png filename=피카.png len=46511>


46511

이메일 관리 – 메일 내용 읽기   
• 본문 텍스트 내용은 message.text_part 혹은 message.html_part로 확인   

• 첨부 파일이 추가된 메일은 멀티파트(multipart) 형식으로 만들어진다.    
멀티파트 형식의 메일은 본문 이외의 추가적인 정보 혹은 첨부 파일 등을 함께 보낼 때 사용하는 형식이다    

• get_payload() – 본문 내용을 읽어온다   
한글은 디코딩 해야 읽을 수 있다 (part.charset 값을 활용하여 디코딩 )   

이메일 전달   
1. 이메일 한 개 가져온다   
2. 가져온 이메일의 본문 내용을 읽어서 전달할 이메일의 메세지로 구성한다     
   제목, 발신자 정보 설정 등    
3. 메일 보내기

In [149]:
import imapclient
import email

imap = imapclient.IMAPClient('imap.gmail.com', ssl=True)
imap.login('', '')
uids = imap.select_folder('[Gmail]/전체보관함', readonly=False)

#uids = imap.search(['FROM', 'fbvl@naver.com'])
uids = imap.search(['ON', date.today()])[:1]  #오늘 받은 메일 한 개 가져오기
print(uids)
rmsgs = imap.fetch(uids, ['BODY[]'])
m = rmsgs[1635][b'BODY[]']   #uid는 검색된 메일 id값
message = email.message_from_bytes(m)  #전달할 메일의 메세지로 구성


[1635]


In [156]:
from email.header import decode_header, Header

message.get('subject')
subject, encoding = decode_header(message.get('subject'))[0]
str(subject, encoding)

'피카.png'

In [157]:
message['from']
decode_header(message['from'])

[(b'\xec\x8b\xac\xec\x84\xb8\xea\xb2\xbd', 'utf-8'),
 (b' <fbvl@naver.com>', None)]

In [159]:
s = ''
for text , encoding in decode_header(message['from']):  
    if not encoding:
        encoding='utf-8'
    s += str(text, encoding)

print(s)

심세경 <fbvl@naver.com>


In [160]:
new_subject = 'Fwd({}): {}'.format(s, str(subject, encoding))
print(new_subject)

Fwd(심세경 <fbvl@naver.com>): 피카.png


In [162]:
from_addr = 'sim11241124@gmail.com'
to_addr = 'fbvl@naver.com'
message.replace_header('from', from_addr)
message.replace_header('to', to_addr)
del message['cc']
del message['bcc']
message.replace_header('subject', Header(new_subject.encode('utf-8'), 'utf-8'))



In [163]:
import smtplib

smtp = smtplib.SMTP('smtp.gmail.com', 587)
smtp.starttls()
smtp.login('', '')
smtp.sendmail(from_addr, to_addr, message.as_bytes())
smtp.quit()

(221, b'2.0.0 closing connection u25sm760162pgk.34 - gsmtp')

### 프로그램 실행, 관리 자동화   

프로그램 자동 실행  
• os.startfile( path ) - 윈도우 파일 탐색기 실행  

• webbrowser.open( url ) - 브라우저 열기  
실행 명령어에 인수가 필요한 경우는 리스트에 묶어서 전달한다   

• subprocess.Popen( program_name ) - 일반 프로그램 자동실행   
실행 경로를 직접 입력해서 실행 시킬 수 있다.    

• 콘솔(터미널)에서 실행한 명령의 텍스트 결과를 받으려면 stdout에 연결된 파이프(pipe)를 이용한다.    
파이프 - 한 프로그램의 출력을 다른 프로그램의 입력으로 연결해주는 연결 프로그램   

In [166]:
import os
os.startfile('c:\\') # 파일 탐색기 열기

In [169]:
import webbrowser
webbrowser.open('https://www.naver.com')  #브라우저 열기

True

In [168]:
from subprocess import Popen
Popen('calc') # 계산기 실행

In [172]:
Popen(['notepad', 'test.txt'])

In [177]:
from subprocess import PIPE

pipe = Popen('dir', shell=True, stdout=PIPE)
print(pipe.stdout.read().decode('cp949')[:284])

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: 5EFA-1BA0

 C:\workspace\basic 디렉터리

2021-04-22  오후 02:47    <DIR>          .
2021-04-22  오후 02:47    <DIR>          ..
2021-04-22  오후 12:34    <DIR>          .ipynb_checkpoints
2021-04-20  오후 03:40            47,709 1.jpg
2021-04-19  오후 01


지정된 시간에 함수 실행     
• sched 모듈.scheduler객체.enter(delay, priority, action, arguement=(), kwargs={}) -실행할 함수 이벤트를 필요한 만큼 등록 ,   
delay(지정된 시간), priority(우선순위), action(지정된 함수 실행)   
• run() - 등록된 이벤트 실행   

스케쥴을 실행해서 5초 후 my_job이 수행하고 그 아래 10초 후에 my_job이 찍히면 동기방식   
하지만 실행대기 queap에서 두 개 모두 예약을 걸고 각각 실행되는 것은 비동기방식

In [180]:
import sys
import sched

def my_job(text="..."):
    print(text)
    sys.stdout.flush()
    
#블로킹 모드로 실행
s = sched.scheduler()
s.enter(5, 1, my_job)                        #5초 후 my_job() 호출 실행, 1은 우선순위
s.enter(10, 1, my_job, argument=('good!',))  #10초 후 my_job() 호출 실행
s.run() 
# Blocking이란 것인데 이 뜻은 앞선 작업의 결과가 나와야 다음 작업 결과를 보여주는 것
#nonblocking은 그결과 나오는 순서대로 보여주는것

print('done')

...
good!
done


지정된 시간에 함수 실행   
• enterabs(절대시간, priority, action, arguement=(), kwargs={}) - 절대 시작 시간을 지정 , priority(우선순위), action(지정된 함수 실행)

In [181]:
import time

s = sched.scheduler()
s.enterabs(time.time()+5, 1, my_job)

Event(time=1619071189.9601858, priority=1, action=<function my_job at 0x000001BFFB80ED30>, argument=(), kwargs={})

지정된 시간에 함수 실행    
• 예약해 놓은 코드를 대기했다가 지정된 시간에 실행하도록 하면서, 다른 필요한 작업을 동시에 실행하게 하는 구조는 논블로킹 모드로서 파이썬에서 동시 실행이 가능한 구조는 스레딩(threading) 기법으로 해결합니다   

• 하나의 프로그램이 동시에 실행 가능한 작은 단위로 나뉘었을 때, 각각을 스레드(thread) – 각 스레드는 상호협력하면서 동시에 실행된다   

• threading모듈.Timer(interval, function, args=None, kwargs=None) - 지정된 시간에 특정 함수를 실행    
interval= 간격, kwargs=키워드      

interval초 이후에 인수 args와 키워드 인수 kwargs를 갖고 function을 실행하게 해주는 타이머를 생성    

• timer객체.start() - 동작을 시작

• timer객체.cancel() - 타이머 중단

• timer객체.run_it_at() - at인수로 함수 func를 시작할 시간을 지정한다.

threading을 이용한 지정된 시간에 함수 실행하기(논블로킹 모드)

In [190]:
import threading
import time

def my_job(text):
    print(text)

threading.Timer(5, my_job, ('good!!',)).start() # 5초 대기 후 실행
threading.Timer(5, my_job, ('bad!!',)).start() # 5초 대기 후 실행
print('other works continued..')  #바로실행
time.sleep(5)

other works continued..
good!!bad!!



In [196]:
import threading
import time

#지정된 시간에 인수로 전달받은 함수를 실행시키는 함수
def run_it_at(at, func, args=()):
    t = time.strptime(at, '%Y-%m-%d %H:%M:%S')
    t = time.mktime(t) # 지정된 시간을 숫자 시간으로 변환
    diff_sec = t - time.time() # 현재 시간과의 차이 => 대기시간(초)
    print(diff_sec)
    threading.Timer(diff_sec, func, args).start()

def my_job(text):
    print(text)

run_it_at('2021-04-22 15:24:00', my_job, ('excellent!!!',))
print('other works continued..')


#sleep은 여러 thread가 동시에 시작해도 다른 thread들이 끝날 떄 까지 기다리는 느낌으로 사용
#main이 끝날 때 같이 끝날 수가 있어서 기다리는 것
time.sleep(120)  

68.5304958820343
other works continued..
excellent!!!


• scheule 모듈 - 어떤 함수를 주기적으로 실행시키기 위한 함수, 객체 제공  
• schedule.every() - 실행 주기 설정

In [200]:
! pip install schedule

In [ ]:
import schedule
import time

def job(msg='I`m working...'):
    print(msg)
    
schedule.every().seconds.do(job)  #1초마다
schedule.every(5).seconds.do(job, ('5 sec',))  #매5초 마다
schedule.every(10).minutes.do(job)  #10분마다
schedule.every().hour.do(job, ('hour',))  #매시간 마다
schedule.every().day.at("10:30").do(job)  #매 10시30분마다
schedule.every(5).to(10).seconds.do( job) # 5초에서 10초 마다 (랜덤으로 선택된다)
schedule.every().wednesday.at("13:15").do( job) # 수요일 13:15 마다

while True:
    schedule.run_pending() # 대기 중인 작업이 있다면 처리한다.
    time.sleep(1)

• 윈도우의 작업 스케줄러 schtasks.exe는 리눅스의 cron에 해당   
• schtasks는 스크립트 자체가 실행 중이지 않아도 예약한 시간이 되면 시스템에서 알아서 스크립트를 시작하게 된다.    
• 예약 작업을 설정할 때 윈도우 GUI 작업 스케줄러를 이용할 수도 있지만, schtasks.exe 명령으로 처리하면서 파이썬으로 등록하고 삭제할 수 있으므로 더욱 유명하게 활용할 수 있다

명령 프롬프트창에서 schtasks 명령으로 현재 등록되어 있는 시스템 예약 작업 정보 확인 할 수 있다

schtasks /create #예약 작업 생성

schtasks /create /tn monitor /sc minute /tr calc #monitor란 이름의 작업은 계산기를 매분 실행

파이썬 스크립트를 실행하려면 파이썬 프로그램의 실행 경로와 스크립트를 함께 지정

schtasks /create /tn monitor /sc minute /tr “C:\ProgramData\Anaconda3\python.exe

c:\workspace\basic\monitor.py”

schtasks /create /tn monitor /sc minute /mo 5 /tr calc # 계산기를 5분마다 실행

schtasks /create /tn monitor /sc daily /st 09:00 /tr calc #매일 아침 9시에 계산기를 실행

schtasks /create /tn monitor /sc weekly /d Mon /st 09:00 /tr calc #매주 월요일 아침 9시에 계산기를 실행

schtasks /delete # 예약 작업 삭제

schtasks /delete /tn monitor /f  # #monitor란 이름의 작업 실행 삭제

윈도우에서 작업 스케줄러 예약 작업 생성 옵션

• /tn : taskname, 작업 이름

• /sc : scheduler, 스케줄 빈도 설정

• /mo : modifier, 반복 주기 값을 지정

• /sd : startdate, 작업 시작 첫 번째 날짜

• /ed : enddate, 작업의 종료 일자

• /st : starttime, 작업 시작 시간

• /et : endtime 작업 종료 시간

• /d : 요일

• /k : kill, 작업 종료 시간이 되면, 실행 중인 프로그램을 강제 종료

• /it : 사용자가 로그온되어 있는 경우에만 실행

• /tr : taskrun, 실행할 프로그램

• /f : force, 작업이 수행되는 중이라도 임무를 수행한다

윈도우에서 작업 스케줄러 예약 - 연습문제      
뉴스 리더: 1시간 간격으로 뉴스 url수집해서 메일로 보내기   
작업 예약 등록 명령  
schtasks /create /tn newreader /sc hourly /mo 1 /tr "c:\workspace\basic\newreader.py"


• 등록한 키워드가 뉴스의 제목이나 요약에 있으면 이메일로 그 소식을 알려주는 프로그램 작성하고, 매시간 작동시키는 파이썬 프로그램 구현 (관심 키워드 : 금리, 환율)

In [205]:
import smtplib
import feedparser
from email.mime.text import MIMEText
from email.header import Header

# 관심 RSS feed 주소를 추가, 변경한다.
rss_feeds = [
'http://file.mk.co.kr/news/rss/rss_30100041.xml',
'http://file.mk.co.kr/news/rss/rss_50300009.xml'
]

# words of interest. 관심 단어
WOI = ['금리', '환율']

out = []
for feed in rss_feeds:
    d = feedparser.parse(feed)
    for entry in d.entries:
        for w in WOI:  #관심 단어가 제목이나 요약에 있는지 살핀다
            if w in entry['title'] or w in entry['summary']:
                s = '* {}{}'.format(entry['title'], entry['link'])
                out.append(s)
                break

message = '\n'.join(out)

if message:
    subject = 'RSS News'
    mail_from = 'sim11241124@gmail.com'
    mail_to = 'fbvl@naver.com'
    id_ = ''  #smtp 서버에 로그인 가능한 메일 id
    pw_ = '' #smtp서버에 로그인 가능한 메일 password
    
    smtp = smtplib.SMTP('smtp.gmail.com', 587)
    smtp.ehlo()
    smtp.starttls()
    smtp.login('', '')
    
    msg = MIMEText(message.encode('utf-8'), _subtype='plain', _charset='utf-8')
    msg['Subject'] = Header(subject.encode('utf-8'), 'utf-8')
    msg['From'] = mail_from
    msg['To'] = mail_to
    
    smtp.sendmail(mail_from, mail_to, msg.as_string())
    smtp.quit()